<center>
    This is Set 2: Files and Formats (Exercises 11-20)
</center>

**Prerequisites**
* Additional package `pyarrow` should be installed.
* Small example values along with the sample dataset [datatableton_sample.zip](https://github.com/vopani/datatableton/blob/main/data/datatableton_sample.zip) will be used for the exercises.

In [1]:
# !pip3 install pyarrow

In [2]:
import numpy as np
import pandas as pd
import pyarrow as pa
from zipfile import ZipFile

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/20 10:26:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [115]:
rdd = spark.sparkContext.parallelize(list(zip(list(range(0, 11)), list("NEVERGIVEUP"))))
dframe = spark.createDataFrame(data=rdd, schema=["v1", "v2"])

dframe_pd = pd.DataFrame({'v1': range(11), 'v2': list("NEVERGIVEUP")})
nparray = np.array([[0, 'C'], [1, 'O'], [2, 'D'], [3, 'E']])
patable = pa.Table.from_pandas(dframe_pd)
dlist = [list(range(4)), ['D', 'A', 'T', 'A']]
ddict = {'x': list(range(6)), 'y': ['P', 'Y', 'T', 'H', 'O', 'N']}
dtup = [(0, 'L'), (1, 'E'), (2, 'A'), (3, 'R'), (4, 'N')]

**Exercise 11: Write `dframe` to a csv named `data1.csv`, to a compressed gz csv named `data.gz`**

In [5]:
dframe.show()

+---+---+
| v1| v2|
+---+---+
|  0|  N|
|  1|  E|
|  2|  V|
|  3|  E|
|  4|  R|
|  5|  G|
|  6|  I|
|  7|  V|
|  8|  E|
|  9|  U|
| 10|  P|
+---+---+



In [6]:
dframe_schema = dframe.schema

In [8]:
dframe.write.csv("data1", header=True, mode="overwrite")

In [9]:
dframe.write.csv("data2", header=True, mode="overwrite", compression="gzip")

**Exercise 12: Read `data1.csv` and assign it to `data_csv`, read `data.gz` and assign it to `data_gz`**

In [10]:
data_csv = spark.read.options(header=True).schema(dframe_schema).csv("data1")
data_csv.show()

+---+---+
| v1| v2|
+---+---+
|  9|  U|
| 10|  P|
|  2|  V|
|  3|  E|
|  6|  I|
|  7|  V|
|  0|  N|
|  1|  E|
|  4|  R|
|  5|  G|
|  8|  E|
+---+---+



In [11]:
data_csv.printSchema()

root
 |-- v1: long (nullable = true)
 |-- v2: string (nullable = true)



In [12]:
data_csv.sort("v1").show()

+---+---+
| v1| v2|
+---+---+
|  0|  N|
|  1|  E|
|  2|  V|
|  3|  E|
|  4|  R|
|  5|  G|
|  6|  I|
|  7|  V|
|  8|  E|
|  9|  U|
| 10|  P|
+---+---+



In [13]:
data_gz = spark.read.options(header=True).schema(dframe_schema).csv("data2")
data_gz.show()

+---+---+
| v1| v2|
+---+---+
|  9|  U|
| 10|  P|
|  2|  V|
|  3|  E|
|  6|  I|
|  7|  V|
|  8|  E|
|  5|  G|
|  0|  N|
|  4|  R|
|  1|  E|
+---+---+



In [14]:
data_gz.printSchema()

root
 |-- v1: long (nullable = true)
 |-- v2: string (nullable = true)



**Exercise 13: Read data from this URL: [https://raw.githubusercontent.com/vopani/datatableton/main/data/datatableton_sample.csv](https://raw.githubusercontent.com/vopani/datatableton/main/data/datatableton_sample.csv) and assign it to `data_url`**

In [15]:
url = "https://raw.githubusercontent.com/vopani/datatableton/main/data/datatableton_sample.csv"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
 
path  = SparkFiles.get('datatableton_sample.csv')
data_url = spark.read.csv("file://" + path, header=True, inferSchema= True, sep = ",")
data_url.show()

+-------------------+----+----+-------+--------+-----+--------+
|          timestamp|user| age|product|category|price|quantity|
+-------------------+----+----+-------+--------+-----+--------+
|2017-01-01 13:22:41|  U1|  21|   Eggs|   Dairy|  2.3|       2|
|2017-05-22 09:54:21|  U2|  41|  Bread|      NA|  0.6|       2|
|2018-11-09 15:00:01|  U3|null| Banana|   Fruit|  0.7|       1|
|2018-12-24 23:03:33|  U1|  21|  Water|   Drink|  0.2|       1|
|2019-03-03 06:21:58|  U4|  19|   Eggs|   Dairy|  2.3|       2|
|2019-06-17 16:13:39|  U5|  25| Grapes|   Fruit|  1.5|       1|
|2019-07-28 21:03:11|  U5|  25|  Bread|      NA|  0.6|       1|
|2019-12-05 04:15:42|  U1|  21| Banana|   Fruit|  0.7|       2|
|2020-02-02 03:45:34|  U3|null| Banana|   Fruit|  0.7|       1|
|2020-03-05 07:09:12|  U4|  19| Grapes|   Fruit|  1.5|       1|
|2020-03-22 19:29:38|  U1|  21|  Water|   Drink|  0.2|       1|
|2020-03-30 09:44:30|  U4|  19|  Water|   Drink|  0.2|       1|
|2020-04-01 13:21:41|  U1|  21| Banana| 

**Exercise 14: Read `users.csv` from `datatableton_sample.zip` and assign it to `data_zip`**

In [16]:
z = ZipFile('datatableton_sample.zip')
csv = [l.decode('utf-8').replace('\n', '') for l in z.open('users.csv').readlines()]


In [36]:
data_zip_rdd = (spark
    .sparkContext
    .parallelize(csv)
)
rows = data_zip_rdd.map(lambda x: x.replace("\r", "").split(",")).collect()
headers = rows[0]
rdd = spark.sparkContext.parallelize(rows[1:])

In [37]:
data_zip = spark.createDataFrame(data=rdd, schema=headers)
data_zip.show()

+--------+-----------+---------+-------+
| ﻿Person|Category ID|Return ID| Region|
+--------+-----------+---------+-------+
|     Amy|       1001|     R004|   West|
|     Max|       1002|     R001|   East|
| William|       1007|     R003|Central|
|   Nancy|       1004|     R006|  South|
|Isabella|       1001|     R006|   West|
|     Tom|       1001|     R005|   East|
|    Mary|       1007|     R005|  South|
|  Carlos|       1008|     R006|   West|
|    Kimi|       1003|     R002|   East|
|   Lando|       1002|     R003|Central|
|   Perez|       1001|     R001|   East|
|   Lewis|       1003|     R001|  South|
|  Monica|       1005|     R002|   West|
|   Carla|       1001|     R007|   East|
+--------+-----------+---------+-------+



**Exercise 15: Create a PySpark DataFrame `data_pd` from the pd.DataFrame `dframe` and create a pd.DataFrame `pd_data` from the PySpark DataFrame `data_pd`**

In [39]:
data_pd = spark.createDataFrame(dframe_pd)
data_pd.show()

+---+---+
| v1| v2|
+---+---+
|  0|  N|
|  1|  E|
|  2|  V|
|  3|  E|
|  4|  R|
|  5|  G|
|  6|  I|
|  7|  V|
|  8|  E|
|  9|  U|
| 10|  P|
+---+---+



In [40]:
pd_data = data_pd.toPandas()
pd_data

,v1,v2
0,0,N
1,1,E
2,2,V
3,3,E
4,4,R
5,5,G
6,6,I
7,7,V
8,8,E
9,9,U


**Exercise 16: Create a PySpark DataFrame `data_np` from the np.array `nparray` and create a np.array `np_data` from the PySpark DataFrame `data_np`**

In [63]:
nparray

array([['0', 'C'],
       ['1', 'O'],
       ['2', 'D'],
       ['3', 'E']], dtype='<U21')

In [82]:
rdd = spark.sparkContext.parallelize(nparray.tolist())
data_np = rdd.toDF()
data_np.show()

+---+---+
| _1| _2|
+---+---+
|  0|  C|
|  1|  O|
|  2|  D|
|  3|  E|
+---+---+



In [84]:
np_data = np.array(data_np.collect())
np_data

array([['0', 'C'],
       ['1', 'O'],
       ['2', 'D'],
       ['3', 'E']], dtype='<U1')

**Exercise 17: Create a PySpark DataFrame `data_ls` from the list `dlist` and create a list `ls_data` from the PySpark DataFrame `data_ls`**

In [89]:
dlist

[[0, 1, 2, 3], ['D', 'A', 'T', 'A']]

In [94]:
rdd = spark.sparkContext.parallelize(list(zip(dlist[0], dlist[1])))
data_ls = rdd.toDF()
data_ls.show()

+---+---+
| _1| _2|
+---+---+
|  0|  D|
|  1|  A|
|  2|  T|
|  3|  A|
+---+---+



In [113]:
ls_data = [data_ls.select(col).rdd.flatMap(lambda x: x).collect() for col in data_ls.columns]
ls_data

[[0, 1, 2, 3], ['D', 'A', 'T', 'A']]

**Exercise 18: Create a PySpark DataFrame `data_dc` from the dictionary `ddict` and create a dictionary `dc_data` from the PySpark DataFrame `data_dc`**

In [127]:
ddict

{'x': [0, 1, 2, 3, 4, 5], 'y': ['P', 'Y', 'T', 'H', 'O', 'N']}

In [139]:
dict_values = list(ddict.values())

rdd = spark.sparkContext.parallelize(list(zip(dict_values[0], dict_values[1])))
data_dc = rdd.toDF(list(ddict.keys()))
data_dc.show()

+---+---+
|  x|  y|
+---+---+
|  0|  P|
|  1|  Y|
|  2|  T|
|  3|  H|
|  4|  O|
|  5|  N|
+---+---+



In [143]:
dc_data = {column:data_dc.select(column).rdd.flatMap(lambda x: x).collect() for column in data_dc.columns}
dc_data

{'x': [[0, 1, 2, 3, 4, 5]], 'y': [['P', 'Y', 'T', 'H', 'O', 'N']]}

**Exercise 19: Create a PySpark DataFrame `data_tp` from the tuples `dtup` and create tuples `tp_data` from the PySpark DataFrame `data_tp`**

In [36]:
dtup

[(0, 'L'), (1, 'E'), (2, 'A'), (3, 'R'), (4, 'N')]

In [35]:
data_tp = pd.DataFrame(dtup)
data_tp

,0,1
0,0,L
1,1,E
2,2,A
3,3,R
4,4,N


In [41]:
tp_data = list(data_tp.to_records(index=False))
tp_data

[(0, 'L'), (1, 'E'), (2, 'A'), (3, 'R'), (4, 'N')]

**Exercise 20: Create a PySpark DataFrame `data_pa` from the pyarrow.Table `patable` and create a pyarrow.Table `pa_data` from the PySpark DataFrame `data_pa`**

In [47]:
patable

pyarrow.Table
v1: int64
v2: string
----
v1: [[0,1,2,3,4,5,6,7,8,9,10]]
v2: [["N","E","V","E","R","G","I","V","E","U","P"]]

In [46]:
data_pa = patable.to_pandas()
data_pa

,v1,v2
0,0,N
1,1,E
2,2,V
3,3,E
4,4,R
5,5,G
6,6,I
7,7,V
8,8,E
9,9,U


In [49]:
pa_data = pa.Table.from_pandas(data_pa)
pa_data

pyarrow.Table
v1: int64
v2: string
----
v1: [[0,1,2,3,4,5,6,7,8,9,10]]
v2: [["N","E","V","E","R","G","I","V","E","U","P"]]

✅ This completes Set 2: Files and Formats (Exercises 11-20)

Original exercises for Datatable package can be found [here](https://github.com/vopani/datatableton)